# 1 - Imports

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
import time

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

## 1.1 - Funções

In [3]:
def url_bs(url):
    page = requests.get(url, headers=headers)
    soup = bs(page.text, 'html.parser')
    return soup

# 2 - Coleta de dados

## 2.1 - Webscraping

In [4]:
# lista de categorias
lista_categoria = []
soup = url_bs('https://books.toscrape.com/')
aux = soup.find('ul', class_='nav nav-list').find_all('a')
aux1 = [p.get_text('href') for p in aux]
for e in aux1:
    aux2 = re.findall('\w{0,}.\S+', e)
    aux2 = ''.join(aux2)
    lista_categoria.append(aux2)

del lista_categoria[0]

# lista das url das categorias
browser = webdriver.Chrome(chrome_options=chrome_options)
browser.get('https://books.toscrape.com')
lista_url_categoria = []
for i in range(1, len(lista_categoria) + 1):
    browser.find_element_by_xpath(f'//*[@id="default"]/div[1]/div/div/aside/div[2]/ul/li/ul/li[{i}]/a').click()
    lista_url_categoria.append(browser.current_url)
    
# Webscraping

# Listas auxiliares
lista_nome = []
lista_preco = []
lista_avaliacao = []
lista_estoque = []
lista_categoria_2 = []

for i in range (len(lista_categoria)):
    categoria = lista_categoria[i]
    url_categoria = lista_url_categoria[i]
    browser.get(url_categoria)
    soup = url_bs(url_categoria)   
    
################  PÁGINA 1 ####################  
################################### NOME DO LIVRO ##############################################
    aux_nome = soup.find_all('h3')
    nome = [p.get_text() for p in aux_nome]
    lista_nome = lista_nome + nome
    
#################################### CATEGORIA #################################################
    aux_categoria = soup.find('form', class_='form-horizontal').find_all('strong')
    aux_categoria_2 = [p.get_text() for p in aux_categoria][0]
    for e in range(int(aux_categoria_2)):
        lista_categoria_2.append(categoria)
    
################################## PREÇO DO LIVRO ##############################################
    aux_preco = soup.find_all('p', class_='price_color')
    preco = [p.get_text() for p in aux_preco]
    lista_preco = lista_preco + preco

################################ AVALIAÇÃO DO LIVRO ############################################
    aux_avaliacao = soup.find_all('p', class_='star-rating')
    avaliacao = [p.get('class')[1] for p in aux_avaliacao]
    lista_avaliacao = lista_avaliacao + avaliacao
    
#################################### ESTOQUE ###################################################
    aux_estoque = soup.find_all('p', class_='instock availability')
    estoque = [p.get_text() for p in aux_estoque]
    lista_estoque = lista_estoque + estoque
    
    # Verificar se há paginação
    paginacao = soup.find('li', class_='current')
    if paginacao:
        aux = soup.find('li', class_='current').get_text()
        aux1 = re.findall('\d{0,}', aux)
        aux2 = list(filter(None, aux1))[1]
        aux2 = int(aux2) - 1

        for p in range(aux2 + 1):
            if p == 0: 
               ######### ENVIA PARA A PÁGINA 2 ##################  
                browser.find_element_by_xpath('//*[@id="default"]/div[1]/div/div/div/section/div[2]/div/ul/li[2]/a').click()
                soup = url_bs(browser.current_url)

            elif (p > 0) & (p != aux2): ######## PÁGINAS 2 ###############                
################################### NOME DO LIVRO ##############################################
                aux_nome = soup.find_all('h3')
                nome = [p.get_text() for p in aux_nome]
                lista_nome = lista_nome + nome
            
################################## PREÇO DO LIVRO ##############################################
                aux_preco = soup.find_all('p', class_='price_color')
                preco = [p.get_text() for p in aux_preco]
                lista_preco = lista_preco + preco
            
################################ AVALIAÇÃO DO LIVRO ############################################
                aux_avaliacao = soup.find_all('p', class_='star-rating')
                avaliacao = [p.get('class')[1] for p in aux_avaliacao]
                lista_avaliacao = lista_avaliacao + avaliacao
            
#################################### ESTOQUE ###################################################
                aux_estoque = soup.find_all('p', class_='instock availability')
                estoque = [p.get_text() for p in aux_estoque]
                lista_estoque = lista_estoque + estoque
            
                ######### PRÓXIMA PÁGINA #######
                browser.find_element_by_xpath('//*[@id="default"]/div[1]/div/div/div/section/div[2]/div/ul/li[3]/a').click()
                soup = url_bs(browser.current_url)

            else: ######## PÁGINA 3 EM DIANTE ###########               
################################### NOME DO LIVRO ##############################################
                aux_nome = soup.find_all('h3')
                nome = [p.get_text() for p in aux_nome]
                lista_nome = lista_nome + nome 
            
################################## PREÇO DO LIVRO ##############################################
                aux_preco = soup.find_all('p', class_='price_color')
                preco = [p.get_text() for p in aux_preco]
                lista_preco = lista_preco + preco

################################ AVALIAÇÃO DO LIVRO ############################################
                aux_avaliacao = soup.find_all('p', class_='star-rating')
                avaliacao = [p.get('class')[1] for p in aux_avaliacao]
                lista_avaliacao = lista_avaliacao + avaliacao
    
#################################### ESTOQUE ###################################################
                aux_estoque = soup.find_all('p', class_='instock availability')
                estoque = [p.get_text() for p in aux_estoque]
                lista_estoque = lista_estoque + estoque

 
browser.close()

data = {'nome':lista_nome, 'categoria':lista_categoria_2, 'preco':lista_preco, 'avaliacao':lista_avaliacao, 'estoque':lista_estoque}
df = pd.DataFrame(data)
df['data'] = datetime.now().strftime('%Y-%m-%d')
df['horario'] = datetime.now().strftime('%H:%M:%S')

# 2 - Feature Engineering

In [5]:
df['estoque'] = df['estoque'].str.extract('(In stock)')
df['preco'] = df['preco'].str.extract('(\d.+)')

df.loc[df['avaliacao'] == 'One', 'avaliacao'] = 1
df.loc[df['avaliacao'] == 'Two', 'avaliacao'] = 2
df.loc[df['avaliacao'] == 'Three', 'avaliacao'] = 3
df.loc[df['avaliacao'] == 'Four', 'avaliacao'] = 4
df.loc[df['avaliacao'] == 'Five', 'avaliacao'] = 5

# 3 - Exportando os dados

In [6]:
df.to_csv('teste.csv', encoding='latin-1', index=False)